In [19]:
import numpy as np
from data_helper import DatasetAndQuerysetHelper
from partition_algorithm import PartitionAlgorithm
import ray

In [20]:
used_dims = [1,2,4]
scale_factor=1
# block_size = int(1000000/scale_factor)
block_size = 10000
max_active_ratio=3
base_path='/home/liupengju/pycharmProjects/NORA_JOIN_SIMULATION/NORA_experiments'


In [21]:
helper = DatasetAndQuerysetHelper(used_dimensions=used_dims, scale_factor=scale_factor)
# helper.maximum_range_percent = 0.2

In [4]:

# helper.generate_dataset_and_save(base_path+'/dataset/lineitem_1.tbl')

In [22]:
dataset, domains = helper.load_dataset(used_dims)
boundary = [interval[0] for interval in domains] + [interval[1] for interval in domains]
print(len(dataset))  # 6001309
print(domains)

6001215
[[1.e+00 2.e+05]
 [1.e+00 1.e+04]
 [1.e+00 5.e+01]]


In [6]:
query_amount=np.array([0.05,0.1,0.2,0.5,1])*1000
print(query_amount)

[  50.  100.  200.  500. 1000.]


In [7]:
result={}
for query_num in query_amount:
    training_set, testing_set = helper.generate_queryset_and_save(int(query_num), queryset_type=3) # 1-1
    cost_res = list()
    # = = = = = Test PartitionAlgorithm (QDT) = = = = =
    pa1 = PartitionAlgorithm()
    pa1.InitializeWithQDT(training_set, len(boundary) // 2, boundary, dataset, data_threshold=block_size)

    # = = = = = Test PartitionAlgorithm (PAW) = = = = =
    pa2 = PartitionAlgorithm()
    # pa2.InitializeWithPAW(training_set, len(boundary)//2, boundary, dataset, block_size,using_beam_search=False, candidate_size = 2, candidate_depth = 2, max_active_ratio=50,strategy = 0)
    pa2.InitializeWithPAW(training_set, len(boundary) // 2, boundary, dataset, block_size, max_active_ratio=max_active_ratio,strategy=1)


    # = = = = = Test PartitionAlgorithm (ME) = = = = =
    pa3 = PartitionAlgorithm()
    # pa3.InitializeWithPAW2(training_set, len(boundary) // 2, boundary, dataset, block_size,using_beam_search=True, candidate_size = 2, candidate_depth = 2,max_active_ratio=50,strategy = 0)
    pa3.InitializeWithNORA(training_set, len(boundary)//2, boundary, dataset, data_threshold = block_size, using_beam_search=True, candidate_size = 2, candidate_depth = 2,using_1_by_1 = True, using_kd = True)
    pa3.partition_tree.set_redundant_partition(queries=training_set,data_threshold=block_size)

    algos = [pa1,pa2,pa3]
    for algo in algos:
        cost_res.append(algo.partition_tree.evaluate_query_cost(training_set, True))
    result_sizes = helper.real_result_size(dataset, training_set)
    cost_res.append(max(sum(result_sizes)/len(result_sizes), block_size))
    result[query_num]=np.round(np.array(cost_res)/dataset.shape[0],8)

print(result)
# {50.0: array([0.00257215, 0.00212724, 0.00198393, 0.00166633]), 
#  100.0: array([0.00247683, 0.00223538, 0.00203675, 0.00166633]), 
#  200.0: array([0.00272878, 0.00243434, 0.00201992, 0.00166633]), 
#  500.0: array([0.00298723, 0.00268362, 0.00204325, 0.00166633]), 
#  1000.0: array([0.00319535, 0.00299739, 0.00212474, 0.00166633])}

Build Time (s): 3.590029716491699
PAW: overlap exists in (extended) MBRs
PAW: overlap exists in (extended) MBRs
Group Split is used!!!
Group Split is used!!!
Group Split is used!!!
PAW: overlap exists in (extended) MBRs
Group Split is used!!!
Group Split is used!!!
Group Split is used!!!
Group Split is used!!!
Build Time (s): 5.947093486785889
Build Time (s): 22.229274034500122
Total logical IOs: 385907
Average logical IOs: 15436
Total logical IOs: 319156
Average logical IOs: 12766
Total logical IOs: 297671
Average logical IOs: 11906
Build Time (s): 5.84882116317749
PAW: overlap exists in (extended) MBRs
Group Split is used!!!
PAW: overlap exists in (extended) MBRs
PAW: overlap exists in (extended) MBRs
PAW: overlap exists in (extended) MBRs
Group Split is used!!!
Group Split is used!!!
Group Split is used!!!
Group Split is used!!!
Group Split is used!!!
Group Split is used!!!
Build Time (s): 8.952588081359863
Build Time (s): 32.39619541168213
Total logical IOs: 743235
Average logical 

Group Split is used!!!
Group Split is used!!!
Group Split is used!!!
Group Split is used!!!
Group Split is used!!!
Build Time (s): 81.52370643615723
Build Time (s): 243.10073590278625
Total logical IOs: 9588328
Average logical IOs: 19176
Total logical IOs: 8994050
Average logical IOs: 17988
Total logical IOs: 6375924
Average logical IOs: 12751
{50.0: array([0.00257215, 0.00212724, 0.00198393, 0.00166633]), 100.0: array([0.00247683, 0.00223538, 0.00203675, 0.00166633]), 200.0: array([0.00272878, 0.00243434, 0.00201992, 0.00166633]), 500.0: array([0.00298723, 0.00268362, 0.00204325, 0.00166633]), 1000.0: array([0.00319535, 0.00299739, 0.00212474, 0.00166633])}


In [14]:
# search dimensionality
helper = DatasetAndQuerysetHelper(scale_factor=scale_factor) # EXAMPLE
c_dataset, c_domains = helper.load_dataset()
query_num=500
result={}
available_dims=[0,1,2,4,5,3,6]
for num_dims in range(2,8):
    # used_dims = [i for i in range(1, num_dims+1)]
    used_dims=available_dims[:num_dims]
    dataset=c_dataset[:,used_dims]
    domains=c_domains[used_dims]
    boundary = [interval[0] for interval in domains]+[interval[1] for interval in domains]
    helper.used_dimensions=used_dims
    training_set, testing_set = helper.generate_queryset_and_save(query_num, queryset_type=3)
    cost_res = list()
   # = = = = = Test PartitionAlgorithm (QDT) = = = = =
    pa1 = PartitionAlgorithm()
    pa1.InitializeWithQDT(training_set, len(boundary) // 2, boundary, dataset, data_threshold=block_size)

    # = = = = = Test PartitionAlgorithm (PAW) = = = = =
    pa2 = PartitionAlgorithm()
    # pa2.InitializeWithPAW(training_set, len(boundary)//2, boundary, dataset, block_size,using_beam_search=False, candidate_size = 2, candidate_depth = 2, max_active_ratio=50,strategy = 0)
    pa2.InitializeWithPAW(training_set, len(boundary) // 2, boundary, dataset, block_size, max_active_ratio=max_active_ratio,strategy=1)

    # = = = = = Test PartitionAlgorithm (ME) = = = = =
    pa3 = PartitionAlgorithm()
    # pa3.InitializeWithPAW2(training_set, len(boundary) // 2, boundary, dataset, block_size,using_beam_search=True, candidate_size = 2, candidate_depth = 2,max_active_ratio=50,strategy = 0)
    pa3.InitializeWithNORA(training_set, len(boundary)//2, boundary, dataset, data_threshold = block_size, using_beam_search=True, candidate_size = 2, candidate_depth = 2,using_1_by_1 = True, using_kd = True)
    pa3.partition_tree.set_redundant_partition(queries=training_set,data_threshold=block_size)

    algos = [pa1,pa2,pa3]
    for algo in algos:
        cost_res.append(algo.partition_tree.evaluate_query_cost(training_set, True))
    result_sizes = helper.real_result_size(dataset, training_set)
    cost_res.append(max(sum(result_sizes)/len(result_sizes), block_size))
    result[str(used_dims)]=np.round(np.array(cost_res)/dataset.shape[0],8)

print(result)
# {'[0, 1]': array([0.00598512, 0.00553938, 0.00433562, 0.00238116]), 
#  '[0, 1, 2]': array([0.00277477, 0.00253665, 0.00202359, 0.00166633]), 
#  '[0, 1, 2, 4]': array([0.00255248, 0.00238468, 0.00195627, 0.00166633]), 
#  '[0, 1, 2, 4, 5]': array([0.00269929, 0.0026458 , 0.00202442, 0.00166633]), 
#  '[0, 1, 2, 4, 5, 3]': array([0.00266813, 0.00261714, 0.00199543, 0.00166633]),
#  '[0, 1, 2, 4, 5, 3, 6]': array([0.00250416, 0.00248233, 0.00196727, 0.00166633])}

Build Time (s): 31.807917833328247
PAW: overlap exists in (extended) MBRs
Group Split is used!!!
PAW: overlap exists in (extended) MBRs
PAW: overlap exists in (extended) MBRs
Group Split is used!!!
Group Split is used!!!
Group Split is used!!!
PAW: overlap exists in (extended) MBRs
Group Split is used!!!
Group Split is used!!!
Group Split is used!!!
Group Split is used!!!
Group Split is used!!!
Group Split is used!!!
Group Split is used!!!
PAW: overlap exists in (extended) MBRs
Group Split is used!!!
PAW: overlap exists in (extended) MBRs
Group Split is used!!!
Group Split is used!!!
Group Split is used!!!
Group Split is used!!!
Group Split is used!!!
Group Split is used!!!
Group Split is used!!!
PAW: overlap exists in (extended) MBRs
Group Split is used!!!
Group Split is used!!!
Group Split is used!!!
Group Split is used!!!
Group Split is used!!!
PAW: overlap exists in (extended) MBRs
Group Split is used!!!
Group Split is used!!!
Build Time (s): 37.769446849823
Build Time (s): 110.433

Group Split is used!!!
PAW: overlap exists in (extended) MBRs
PAW: overlap exists in (extended) MBRs
PAW: overlap exists in (extended) MBRs
Group Split is used!!!
Group Split is used!!!
PAW: overlap exists in (extended) MBRs
Group Split is used!!!
Group Split is used!!!
PAW: overlap exists in (extended) MBRs
Group Split is used!!!
Group Split is used!!!
Group Split is used!!!
PAW: overlap exists in (extended) MBRs
Group Split is used!!!
Group Split is used!!!
PAW: overlap exists in (extended) MBRs
Group Split is used!!!
Build Time (s): 122.21494197845459
Build Time (s): 427.7589387893677
Total logical IOs: 3757110
Average logical IOs: 15028
Total logical IOs: 3724255
Average logical IOs: 14897
Total logical IOs: 2951606
Average logical IOs: 11806
{'[0, 1]': array([0.00598512, 0.00553938, 0.00433562, 0.00238116]), '[0, 1, 2]': array([0.00277477, 0.00253665, 0.00202359, 0.00166633]), '[0, 1, 2, 4]': array([0.00255248, 0.00238468, 0.00195627, 0.00166633]), '[0, 1, 2, 4, 5]': array([0.0026

In [34]:
ray.init(num_cpus=10)

2022-07-03 15:10:04,523	INFO services.py:1173 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '10.77.110.133',
 'raylet_ip_address': '10.77.110.133',
 'redis_address': '10.77.110.133:6379',
 'object_store_address': '/tmp/ray/session_2022-07-03_15-10-03_837441_128231/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-07-03_15-10-03_837441_128231/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2022-07-03_15-10-03_837441_128231',
 'metrics_export_port': 63890,
 'node_id': '6c96ac8c356d28d03403f44d96da503f5d095109'}

In [41]:
@ray.remote(num_returns=1)
def compute_partition_result_by_range_percent(percent):
    helper.maximum_range_percent = percent
    training_set, testing_set = helper.generate_queryset_and_save(query_num, queryset_type=3)
    print(len(training_set))
    
   # = = = = = Test PartitionAlgorithm (QDT) = = = = =
    pa1 = PartitionAlgorithm()
    pa1.InitializeWithQDT(training_set, len(boundary) // 2, boundary, dataset, data_threshold=block_size)

    # = = = = = Test PartitionAlgorithm (PAW) = = = = =
    pa2 = PartitionAlgorithm()
    # pa2.InitializeWithPAW(training_set, len(boundary)//2, boundary, dataset, block_size,using_beam_search=False, candidate_size = 2, candidate_depth = 2, max_active_ratio=50,strategy = 0)
    pa2.InitializeWithPAW(training_set, len(boundary) // 2, boundary, dataset, block_size, max_active_ratio=max_active_ratio,strategy=1)

    # = = = = = Test PartitionAlgorithm (ME) = = = = =
    pa3 = PartitionAlgorithm()
    # pa3.InitializeWithPAW2(training_set, len(boundary) // 2, boundary, dataset, block_size,using_beam_search=True, candidate_size = 2, candidate_depth = 2,max_active_ratio=50,strategy = 0)
    pa3.InitializeWithNORA(training_set, len(boundary)//2, boundary, dataset, data_threshold = block_size, using_beam_search=True, candidate_size = 2, candidate_depth = 2,using_1_by_1 = True, using_kd = True)
    #     adapt for query max range <= \thera
    pa3.partition_tree.set_redundant_partition(queries=training_set,data_threshold=block_size)
    algos = [pa1,pa2,pa3]
    cost_res = list()
    for algo in algos:
        cost_res.append(algo.partition_tree.evaluate_query_cost(training_set, True))
    result_sizes = helper.real_result_size(dataset, training_set)
    cost_res.append(max(sum(result_sizes)/len(result_sizes), block_size))
    print("Have finish ",percent)
    return cost_res



In [42]:
# vary query max range

query_num=200
result={}
# range_percent_list=[0.01,0.05,0.1,0.2,0.5]
range_percent_list=[0.3,0.4]
chunks=[]
for percent in range_percent_list:
    chunks.append(compute_partition_result_by_range_percent.remote(percent))
last_ids=chunks.copy()
while len(last_ids):
    done_id, last_ids = ray.wait(last_ids)
#     print(done_id,last_pa_ids)
chunks_list=[ray.get(item) for item in chunks]
cnt=0
for cost_res in chunks_list:
    result[str(range_percent_list[cnt])]=np.round(np.array(cost_res)/dataset.shape[0],8)
    cnt+=1
print(result)

# {'0.01': array([0.00272878, 0.00220839, 0.0019536 , 0.00166633]),
#  '0.05': array([0.00249899, 0.00225171, 0.00204109, 0.00166633]),
#  '0.1': array([0.00275777, 0.0023087 , 0.00200376, 0.00166633]), 
#  '0.2': array([0.00449059, 0.00391121, 0.0024845 , 0.00166633]),
# {'0.3': array([0.00791806, 0.00721471, 0.00444077, 0.00245527]), 
#  '0.4': array([0.01559784, 0.01476984, 0.01099694, 0.00683271])}

#  '0.5': array([0.02476499, 0.02384151, 0.01848809, 0.01150277]),
# '0.7': array([0.04880145, 0.04815175, 0.04736041, 0.0285861 ]),
#     '1': array([0.10810294, 0.10739059, 0.10643795, 0.06912556

2022-07-03 16:21:16,589	WARNING worker.py:1034 -- Warning: The remote function __main__.compute_partition_result_by_range_percent has size 144032063 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.


(pid=315177) 100
(pid=315178) 100
(pid=315178) Build Time (s): 28.49252223968506
(pid=315177) Build Time (s): 27.9755916595459
(pid=315178) PAW: overlap exists in (extended) MBRs
(pid=315177) Group Split is used!!!
(pid=315177) Group Split is used!!!
(pid=315177) Group Split is used!!!
(pid=315178) Group Split is used!!!
(pid=315177) PAW: overlap exists in (extended) MBRs
(pid=315177) PAW: overlap exists in (extended) MBRs
(pid=315177) PAW: overlap exists in (extended) MBRs
(pid=315177) Group Split is used!!!
(pid=315177) Group Split is used!!!
(pid=315177) Group Split is used!!!
(pid=315178) Group Split is used!!!
(pid=315177) PAW: overlap exists in (extended) MBRs
(pid=315178) Group Split is used!!!
(pid=315178) Group Split is used!!!
(pid=315177) Group Split is used!!!
(pid=315177) Group Split is used!!!
(pid=315177) Group Split is used!!!
(pid=315177) Group Split is used!!!
(pid=315178) Group Split is used!!!
(pid=315177) Group Split is used!!!
(pid=315178) Group Split is used!!!
(

In [10]:
ray.shutdown()
